In [0]:
#spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

Out[2]: '0715-020554-xcqbg7l6'

In [0]:
filename = dbutils.widgets.get("filename")

In [0]:
dbutils.fs.unmount("/mnt/sales")

dbutils.fs.mount(
    source = 'wasbs://sales@projectsa100.blob.core.windows.net/',
    mount_point = '/mnt/sales',
    extra_configs = {'fs.azure.account.key.projectsa100.blob.core.windows.net':'4FycUL/gENzmUuoXjSu1T1NyLUtacIuUV5JaYOqod6f8BsT0raUQhGe7Ot3I78mhdBNRHJuf3M4L+AStG85g8g=='}
)


/mnt/sales has been unmounted.
Out[16]: True

In [0]:
%fs

ls /mnt/sales/landing

path,name,size,modificationTime
dbfs:/mnt/sales/landing/orders.csv,orders.csv,224,1720452598000


In [0]:
orders_df = spark.read.csv("/mnt/sales/landing/{}".format(filename), header=True , inferSchema=True)

In [0]:
display(orders_df)

order_id,order_date,customer_id,order_status
1111111,2013-07-25T00:00:00.000+0000,11599,CLOSED
2222222,2013-07-25T00:00:00.000+0000,256,PENDING_PAYMENT
3333333,2013-07-25T00:00:00.000+0000,12111,COMPLETE
4444444,2013-07-25T00:00:00.000+0000,8827,CLOSED


In [0]:
errorFlg = False
orders_count = orders_df.count()
print(orders_count)
orders_dist = orders_df.select("order_id").distinct().count()
print(orders_dist)
if orders_count != orders_dist:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv("/mnt/sales/landing/{}".format(filename), "/mnt/sales/discarded")
    dbutils.notebook.exit('{"error_Flg": "true" , "error_Msg": "order_id is repeating"}')

orders_df.createOrReplaceTempView("orders")

4
4


In [0]:
dbServer = 'sql-server-12345'
dbPort = '1433'
dbName = 'project-sql-db'
dbUser = 'sql-user'
dbPassword = 'sql-login-pass-kv'
databricksScope = 'databricks_secret_scope'

In [0]:
connectionurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbPort, dbName, dbUser)

In [0]:
dbPassword = dbutils.secrets.get(scope = databricksScope, key = 'sql-login-pass-kv')

In [0]:
connectionProperties = {
'password': dbPassword,
'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

In [0]:
validstatusDf = spark.read.jdbc( url = connectionurl, table = 'dbo.valid_order_status', properties = connectionProperties)

In [0]:
display(validstatusDf)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELED
PENDING_PAYMENT


In [0]:
validstatusDf.createOrReplaceTempView('valid_status')

In [0]:

invalidRowDf = spark.sql("select * from orders where order_status not in (select * from valid_status)")

In [0]:
display(invalidRowDf)

order_id,order_date,customer_id,order_status


In [0]:
if invalidRowDf.count() > 0:
    errorFlg = True
if errorFlg :
    dbutils.fs.mv("/mnt/sales/landing/{}".format(filename), "/mnt/sales/discarded")
    dbutils.notebook.exit('{"error_Flg": "true" , "error_Msg": "invalid order status found"}')
else:
    dbutils.fs.mv("/mnt/sales/landing/{}".format(filename), "/mnt/sales/staging")

In [0]:
orderitemsDF = spark.read.csv('/mnt/sales/order_items/subashmb1998/Project-capstown/main/order_items_neworders_new.csv', header= True , inferSchema= True)
display(orderitemsDF)
orderitemsDF.createOrReplaceTempView("order_items")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4336559174083797> in <cell line: 1>()
----> 1 orderitemsDF = spark.read.csv('/mnt/sales/order_items/order_items.csv', header= True , inferSchema= True)

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, positiveInf, negativeInf, dateFormat, timestampFormat, maxColumns, maxCharsPerColumn, maxMalformedLogPerPartition

In [0]:
customersDF = spark.read.jdbc( url = connectionurl, table = 'dbo.customers', properties = connectionProperties)
display(customersDF)
customersDF.createOrReplaceTempView("customers")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4448963473689764> in <cell line: 1>()
----> 1 customersDF = spark.read.jdbc( url = connectionurl, table = 'dbo.customers', properties = connectionProperties)
      2 display(customersDF)
      3 customersDF.createOrReplaceTempView("customers")

NameError: name 'connectionurl' is not defined

In [0]:
orders_df = spark.read.csv("/mnt/sales/staging/{}".format(filename), header=True , inferSchema=True)
orders_df.createOrReplaceTempView("orders")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2590831560303960> in <cell line: 1>()
----> 1 orders_df = spark.read.csv("/mnt/sales/staging/{}".format(filename), header=True , inferSchema=True)
      2 orders_df.createOrReplaceTempView("orders")

NameError: name 'filename' is not defined

In [0]:
resultDF1 = spark.sql("select customers.customer_id, customers.customer_fname, customers.customer_lnam, customers.customer_city, customers.customer_state, customers.customer_zipcode, count(order_id) as num_orders_placed, round(sum(order_item_subtotal),2) as total_amount from customers, orders, order_items where customers.customer_id = orders.customer_id and orders.order_id = order_items.order_item_order_id group by customers.customer_id, customers.customer_fname, customers.customer_lnam, customers.customer_city, customers.customer_state, customers.customer_zipcode order by total_amount desc")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4448963473689767> in <cell line: 1>()
----> 1 spark.sql("select customers.customer_id, customers.customer_customer_fname, customers.customer_lname, customers.customer_city, customers.customer_state, customers.customers_zipcode, count(order_id) as num_orders_placed, round(sum(order_item_subtotal),2) as total_amount from customers, orders, order_items where customers.customer_id = orders.customer_id and orders_order_id = order_items.order_item_order_id group by ustomers.customer_id, customers.customer_customer_fname, customers.customer_lname, customers.customer_city, customers.customer_state, customers.customers_zipcode order by total_amount desc")

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 

In [0]:
display(resultDF1)
resultDF1.write.jdbc(url = connectionurl, table = 'dbo.sales_reporting', properties = connectionProperties, mode = 'overwrite')

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4448963473689768> in <cell line: 1>()
----> 1 display(resultDF1)
      2 resultDF1.spark.write.jdbc(url = connectionurl, table = 'dbo.sales_reporting', properties = connectionProperties, mode = 'overwrite')

NameError: name 'resultDF1' is not defined